In [ ]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from google.colab import drive

# Mount Google Drive
drive.mount('/content/drive')

# Set the path to the main folder containing subfolders for each class in your Google Drive
data_directory = '/content/drive/MyDrive/CNN_PROTANI'

# Set the image dimensions and batch size
image_width, image_height = 128, 128
batch_size = 32

# Create separate ImageDataGenerator objects for training and validation data
train_datagen = ImageDataGenerator(
    rescale=1.0/255.0,  # Normalize pixel values to [0, 1]
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    validation_split=0.2  # Splitting 20% of the data as validation set
)

# Generate training data from the images in the subfolders
train_data_generator = train_datagen.flow_from_directory(
    directory=data_directory,
    target_size=(image_width, image_height),
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=True,
    subset='training'  # Subset of the data for training
)

# Generate validation data from the images in the subfolders
validation_data_generator = train_datagen.flow_from_directory(
    directory=data_directory,
    target_size=(image_width, image_height),
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=True,
    subset='validation'  # Subset of the data for validation
)

# Get the number of classes
num_classes = len(train_data_generator.class_indices)

# Define the CNN model with dropout layers
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(image_width, image_height, 3)),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Dropout(0.25),  # Dropout layer with dropout rate of 0.25
    tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Dropout(0.25),  # Dropout layer with dropout rate of 0.25
    tf.keras.layers.Conv2D(128, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Dropout(0.25),  # Dropout layer with dropout rate of 0.25
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dropout(0.5),  # Dropout layer with dropout rate of 0.5
    tf.keras.layers.Dense(num_classes, activation='softmax')
])

# Compile the model
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Train the model with validation
model.fit(
    train_data_generator,
    epochs=10,
    validation_data=validation_data_generator
)

# Save the trained model
model.save('/content/drive/MyDrive/CNN_PROTANI/trained_model.h5')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Found 5663 images belonging to 5 classes.
Found 1414 images belonging to 5 classes.
Epoch 1/10
177/177 [==============================] - 187s 1s/step - loss: 1.1944 - accuracy: 0.5230 - val_loss: 0.8702 - val_accuracy: 0.6761
Epoch 2/10
177/177 [==============================] - 179s 1s/step - loss: 0.7581 - accuracy: 0.7275 - val_loss: 0.6582 - val_accuracy: 0.7638
Epoch 3/10
177/177 [==============================] - 185s 1s/step - loss: 0.5804 - accuracy: 0.7976 - val_loss: 0.4473 - val_accuracy: 0.8373
Epoch 4/10
177/177 [==============================] - 184s 1s/step - loss: 0.4273 - accuracy: 0.8495 - val_loss: 0.3245 - val_accuracy: 0.8798
Epoch 5/10
177/177 [==============================] - 176s 994ms/step - loss: 0.3196 - accuracy: 0.8872 - val_loss: 0.2559 - val_accuracy: 0.8861
Epoch 6/10
177/177 [==============================] - 182s 1s/step - 

In [ ]:
model.save('/content/drive/MyDrive/CNN_PROTANI/trained_model_1.h5')

In [ ]:
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

with open ("model_cnn_new.tflite", 'wb') as f:
  f.write(tflite_model)

In [ ]:
import cv2
import numpy as np
import urllib.request
from keras.models import load_model

model = load_model('/content/drive/MyDrive/CNN_PROTANI/trained_model_1.h5')

model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

url = 'https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcTONl4XyRJlpfXPCJW_B4w__6ADgTF49TZ_Nz3hrj6AeO7dZ-iohf51vNvs-8cggAZ8AIs&usqp=CAU'  # Replace with the URL of your image
with urllib.request.urlopen(url) as response:
    img_array = np.asarray(bytearray(response.read()), dtype=np.uint8)
    img = cv2.imdecode(img_array, cv2.IMREAD_COLOR)

img = cv2.resize(img, (128, 128))
img = np.reshape(img, [1, 128, 128, 3])

classes = model.predict(img)

print(classes)


1/1 [==============================] - 0s 95ms/step
[[0.0000000e+00 1.4123285e-26 1.0000000e+00 0.0000000e+00 0.0000000e+00]]
